It is clear that to train this kind of models we need contiguous in time images.
We need to figure out a nice way to do this, we cannot simply take the \(k\) previous images because there are some temporal holes between a rainfall and another.
We could give an id to the storms, divide in sequences the storms, which means that when we meet 2 images that are more than 5 minutes temporally spaced we consider the 2 images belonging to different storms. then, in training and choosing epochs we choose randomly one storm and randomly again we choose \(k\) contiguous images of that storm to predict the \(k\)-th.

In [1]:
import requests
import json
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd

In [6]:
id_df = pd.read_csv('../data/id_df_SRI_jpg_radar_1.csv')

# Convert 'time' columns to time, format is hour:minute:second
id_df['date_time'] = pd.to_datetime(id_df['date_time'], format='%Y-%m-%d %H:%M:%S')

id_df['time_diff'] = id_df.iloc[::-1]['date_time'].diff()

# Set 'storm' column based on the condition
id_df['storm'] = id_df['time_diff'] < pd.Timedelta(minutes=11) # select only images with 5 minutes interval

# Set the first row to False since there's no previous row to compare
id_df.loc[-1, 'storm'] = False

# Drop the 'time_diff' column
id_df = id_df.drop(columns='time_diff')

print(len(id_df[id_df['storm']]))

18408


In [8]:
base_url = "https://monitor.protezionecivile.fvg.it/api"

def get_image_product(product_id, file_name):
    endpoint = f"/products/{product_id}"
    url = base_url + endpoint
    headers = {
    "accept": "image/jpeg",
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        with open(file_name, "wb") as file:
            file.write(response.content)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

In [15]:
get_image_product(1990053, "../data/images/1990053.jpg")

In [ ]:
pd.set_option('display.max_rows', None)

id_df_storm = id_df[id_df['storm']]
display(id_df_storm)